In [1]:
%run ./schema_library

In [2]:
# Import Section
from pyspark.sql import functions as psf
from pyspark.sql.types import *

### JSON Struct Array Parsing

In [4]:
def recursive_struct_flattener(_df):
  i = 1
  #if (i < 10):
  print(f"--------- START recursive_struct_flattener {i} ----------")
  flat_cols = [c[0] for c in _df.dtypes if c[1][:6] not in ('struct','array<', 'map')]
  nested_cols = [c[0] for c in _df.dtypes if c[1][:6] == 'struct']
  array_or_map_cols = [c[0] for c in _df.dtypes if (c[1][:6] == 'array<') or ('map' in c[1])]
  # Flatten Struct Types  ---------------------------------------
  if len(nested_cols) > 0:
    print("flat fields: ")
    print(flat_cols)
    print("Struct fields: ")
    print(nested_cols)
    print("Array or Map fields: ")
    print(array_or_map_cols)
    #print("----------------------------------- ")
    nc_select = [nc+'.*' for nc in nested_cols]
    #print("Flattening these Structs: ")
    #print(nc_select)
    nc_select_w_alias = [psf.col(nc+'.'+c).alias(nc+'_'+c) for nc in nested_cols for c in _df.select(nc+'.*').columns]
    #Actual Flattening of Struct Types
    _df = _df.select(flat_cols + nc_select_w_alias + array_or_map_cols)
    #print(_df.printSchema())

  # Flatten Array Types -----------------------------------------
  if len(array_or_map_cols) > 0:
    #return _df
    #handle arrayType here
    #print("Print Schema BEFORE Exploding Array ----")
    #print(_df.printSchema())
    array_col = array_or_map_cols[0]
    #print(f"Array col to be exploded: {array_col}")
    _df = _df.withColumn(f"{array_col}_exploded", psf.explode(psf.col(f"{array_col}"))).drop(psf.col(array_col))
    #print("Print Schema AFTER Exploding Array ----")
    #print(_df.printSchema())

  i = i+1
  # Recursion -------------------------------------------------------------------
  #Check if we still have STRUCT or ARRAY or MAP Type left in DF ----------------
  struct_cols = [c[0] for c in _df.dtypes if c[1][:6] == 'struct']
  array_or_map_cols = [c[0] for c in _df.dtypes if (c[1][:6] == 'array<') or ('map' in c[1])]
  # If these COMPOSITE Data Types are left:
  if (len(struct_cols) > 0) or (len(array_or_map_cols) > 0):
    _df = recursive_struct_flattener(_df)
    print("Calling self recursively")
  else:
    print("completed flattening of Struct in DF column ")
    #print(_df.printSchema())
    
  return _df

##### 1st set of Test Data
##### 3 level deep of JSON data

In [6]:
#Define Raw Text JSON Data with Primary Key values
data = [(1,'''{"CompanyProfile":{"PrimaryAddress":{"AddressLine1":"5001 Statesman Dr","AddressLine2":"","City":"Irving","State":"","Country":"US","ZipCode":"75063","County":"county save"},"ProfileBillingAddress":{"AddressLine1":"5001 Statesman Dr","AddressLine2":"","City":"Irving","State":"","Country":"US","ZipCode":"75063","County":"county save"},"BillingPhone":"7222567865","BillingEmail":"sowmya.simhadri@gmail.com","ProfilePhone":"7222567864","ProfileFaxNumber":"7222563375","ProfileEmail":"sowmya.simhadri@gmail.com","ConsolidationType":1,"BillingAddressType":1},"NotificationEmails":{"GettingStartedEmails":["sowmya.simhadri@gmail.com"],"RenewalAndTrialEmails":["sowmya.simhadri@gmail.com"],"ChannelsAndAuthorizationEmails":["sowmya.simhadri@gmail.com"],"SupportEmails":["sowmya.simhadri@gmail.com"],"BillingEmails":["sowmya.simhadri@gmail.com"],"SignatureEmail":["sowmya.simhadri@gmail.com"]},"BrandCustomization":{"ProfileLogoExternalFileName":"","HeaderBackgroundColor":"#aaebff","HeaderTextColor":"#aaebff","SeparatorLineColor":"#aaebff"},"StatementSettings":{"SharedStatementType":1,"DefaultRemark":""}}''')]

#Define the JSON Schema
profile_profileSetting_schema = StructType([
  StructField("CompanyProfile", StructType([
    StructField("PrimaryAddress", StructType([
      StructField("AddressLine1", StringType()),
      StructField("AddressLine2", StringType()),
      StructField("City", StringType()),
      StructField("State", StringType()),
      StructField("Country", StringType()),
      StructField("ZipCode", StringType()),
      StructField("County", StringType())
    ])),
    StructField("ProfileBillingAddress", StructType([
      StructField("AddressLine1", StringType()),
      StructField("AddressLine2", StringType()),
      StructField("City", StringType()),
      StructField("State", StringType()),
      StructField("Country", StringType()),
      StructField("ZipCode", StringType()),
      StructField("County", StringType())
    ])),
    StructField("BillingPhone", StringType()),
    StructField("BillingEmail", StringType()),
    StructField("ProfilePhone", StringType()),
    StructField("ProfileFaxNumber", StringType()),
    StructField("ProfileEmail", StringType()),
    StructField("ConsolidationType", StringType()),
    StructField("BillingAddressType", StringType())
  ])),
  StructField("NotificationEmails", StructType([
    StructField("GettingStartedEmails", ArrayType(StringType())),
    StructField("RenewalAndTrialEmails", ArrayType(StringType())),
    StructField("ChannelsAndAuthorizationEmails", ArrayType(StringType())),
    StructField("SupportEmails", ArrayType(StringType())),
    StructField("BillingEmails", ArrayType(StringType())),
    StructField("SignatureEmail", ArrayType(StringType()))
  ])),
  StructField("BrandCustomization", StructType([
    StructField("ProfileLogoExternalFileName", StringType()),
    StructField("HeaderBackgroundColor", StringType()),
    StructField("HeaderTextColor", StringType()),
    StructField("SeparatorLineColor", StringType())
  ])),
  StructField("StatementSettings", StructType([
    StructField("SharedStatementType", IntegerType()),
    StructField("DefaultRemark", StringType())
  ]))
])

##### 2nd set of Test Data
##### Array of JSON data

In [8]:
# cart_cartProducts JSON

data = [(1,'''[{"ProductName":"Kenexa Sites Behavior One Time","CartProductPromotion":null,"ProductConfigPromotions":null,"ProductKey":"PDMSKHJDVLYQ6U","IsConfigurable":false,"IsConfigured":false,"ProductProvisionType":0,"VendorKey":null,"ProductConfigSettingTemplateKey":null,"ProductConfigKey":"PLY3AAEKLA2FLB","ParentConfigKey":null,"VendorConfigKey":null,"Quantity":1,"PromoCodes":null,"Price":1060.8,"CustomerKey":null,"CustomerDomainPrefix":null,"CustomerContactId":"8ab2bf7f-b0d9-4dd9-a193-4f96eb963001","ParentOrderLineId":null,"ParentVendorSubscriptionId":null,"BillingFrequency":5,"BillingType":1,"DueToday":1060.8,"originalPrice":1060.8,"RemainingVendorSettingsName":null,"Trial":null,"Services":[{"ServiceKey":"SV7OXLW4CXOCTB","VendorServiceKey":"D13H3LL-1","VendorSubscriptionKey":null,"Name":"Kenexa Sites Behavior","VendorProvisionResponse":null,"ProvisionStatus":1,"SubscriptionStatus":0,"BillingFrequency":5,"CreatedOnUtc":null,"UpdatedOnUtc":null,"Quantity":1.0,"Price":1060.8,"RateCardId":1010939,"Margin":0.0,"DefaultQuantity":null,"Cost":1060.8,"ProvisionDate":null,"ParentServiceKey":null,"ServiceConfiguration":{"QuestionText":null,"QuestionNumber":null,"IsQuestionRequired":null,"OptionText":null,"OptionNumber":null,"MaxAllowedServices":null,"NextAction":null,"NextActionQuestion":null},"IsDummy":null,"VendorKey":null}],"ProfileId":309,"FloorPlanId":null,"Currency":"USD","ProductSettings":null,"CustomerSettings":null,"ResellerSettings":null}]''')]

schema = ArrayType(StructType([
  StructField("ProductName", StringType()),
  StructField("CartProductPromotion", StringType()),
  StructField("ProductConfigPromotions", StringType()),
  StructField("IsConfigurable", BooleanType()),
  StructField("IsConfigured", BooleanType()),
  StructField("ProductProvisionType", StringType()),
  StructField("VendorKey", StringType()),
  StructField("ProductConfigSettingTemplateKey", StringType()),
  StructField("ProductConfigKey", StringType()),
  StructField("ParentConfigKey", StringType()),
  StructField("VendorConfigKey", StringType()),
  StructField("Quantity", IntegerType()),
  StructField("PromoCodes", StringType()),
  StructField("Price", FloatType()),
  StructField("CustomerKey", StringType()),
  StructField("CustomerDomainPrefix", StringType()),
  StructField("CustomerContactId", StringType()),
  StructField("ParentOrderLineId", StringType()),
  StructField("ParentVendorSubscriptionId", StringType()),
  StructField("BillingFrequency", IntegerType()),
  StructField("BillingType", IntegerType()),
  StructField("DueToday", FloatType()),
  StructField("originalPrice", FloatType()),
  StructField("RemainingVendorSettingsName", StringType()),
  StructField("Trial", StringType()),
  StructField("Services", ArrayType(StructType([
    StructField("ServiceKey", StringType()),
    StructField("VendorServiceKey", StringType()),
    StructField("VendorSubscriptionKey", StringType()),
    StructField("Name", StringType()),
    StructField("VendorProvisionResponse", StringType()),
    StructField("ProvisionStatus", IntegerType()),
    StructField("SubscriptionStatus", IntegerType()),
    StructField("BillingFrequency", IntegerType()),
    StructField("CreatedOnUtc", StringType()),
    StructField("UpdatedOnUtc", StringType()),
    StructField("Quantity", FloatType()),
    StructField("Price", FloatType()),
    StructField("RateCardId", LongType()),
    StructField("Margin", FloatType()),
    StructField("DefaultQuantity", StringType()),
    StructField("Cost", FloatType()),
    StructField("ProvisionDate", StringType()),
    StructField("ParentServiceKey", StringType()),
    StructField("ServiceConfiguration", StructType([
      StructField("QuestionText", StringType()),
      StructField("QuestionNumber", StringType()),
      StructField("IsQuestionRequired", StringType()),
      StructField("OptionText", StringType()),
      StructField("OptionNumber", StringType()),
      StructField("MaxAllowedServices", StringType()),
      StructField("NextAction", StringType()),
      StructField("NextActionQuestion", StringType())
    ])),
    StructField("IsDummy", StringType()),
    StructField("VendorKey", StringType())
  ]))),
  StructField("ProfileId", IntegerType()),
  StructField("FloorPlanId", StringType()),
  StructField("Currency", StringType()),
  StructField("ProductSettings", StringType()),
  StructField("CustomerSettings", StringType()),
  StructField("ResellerSettings", StringType())
]))

In [9]:
# Testing JsonSchemas() class
# -------------------------------------------
js = JsonSchemas()
schema2 = js.get_table_level_schema("profile")
print(type(schema2))
print(schema2)

print(type(schema2["profilesetting"]))
print(schema2["profilesetting"])

JsonSchemas class instantiated
<class 'collections.OrderedDict'>
OrderedDict([('profilesetting', StructType(List(StructField(CompanyProfile,StructType(List(StructField(PrimaryAddress,StructType(List(StructField(AddressLine1,StringType,true),StructField(AddressLine2,StringType,true),StructField(City,StringType,true),StructField(State,StringType,true),StructField(Country,StringType,true),StructField(ZipCode,StringType,true),StructField(County,StringType,true))),true),StructField(ProfileBillingAddress,StructType(List(StructField(AddressLine1,StringType,true),StructField(AddressLine2,StringType,true),StructField(City,StringType,true),StructField(State,StringType,true),StructField(Country,StringType,true),StructField(ZipCode,StringType,true),StructField(County,StringType,true))),true),StructField(BillingPhone,StringType,true),StructField(BillingEmail,StringType,true),StructField(ProfilePhone,StringType,true),StructField(ProfileFaxNumber,StringType,true),StructField(ProfileEmail,StringType,true),StructField(ConsolidationType,IntegerType,true),StructField(BillingAddressType,IntegerType,true))),true),StructField(NotificationEmails,StructType(List(StructField(GettingStartedEmails,ArrayType(StringType,true),true),StructField(RenewalAndTrialEmails,ArrayType(StringType,true),true),StructField(ChannelsAndAuthorizationEmails,ArrayType(StringType,true),true),StructField(SupportEmails,ArrayType(StringType,true),true),StructField(BillingEmails,ArrayType(StringType,true),true),StructField(SignatureEmail,ArrayType(StringType,true),true))),true),StructField(BrandCustomization,StructType(List(StructField(ProfileLogoExternalFileName,StringType,true),StructField(HeaderBackgroundColor,StringType,true),StructField(HeaderTextColor,StringType,true),StructField(SeparatorLineColor,StringType,true))),true),StructField(StatementSettings,StructType(List(StructField(SharedStatementType,IntegerType,true),StructField(DefaultRemark,StringType,true))),true))))])
<class 'pyspark.sql.types.StructType'>
StructType(List(StructField(CompanyProfile,StructType(List(StructField(PrimaryAddress,StructType(List(StructField(AddressLine1,StringType,true),StructField(AddressLine2,StringType,true),StructField(City,StringType,true),StructField(State,StringType,true),StructField(Country,StringType,true),StructField(ZipCode,StringType,true),StructField(County,StringType,true))),true),StructField(ProfileBillingAddress,StructType(List(StructField(AddressLine1,StringType,true),StructField(AddressLine2,StringType,true),StructField(City,StringType,true),StructField(State,StringType,true),StructField(Country,StringType,true),StructField(ZipCode,StringType,true),StructField(County,StringType,true))),true),StructField(BillingPhone,StringType,true),StructField(BillingEmail,StringType,true),StructField(ProfilePhone,StringType,true),StructField(ProfileFaxNumber,StringType,true),StructField(ProfileEmail,StringType,true),StructField(ConsolidationType,IntegerType,true),StructField(BillingAddressType,IntegerType,true))),true),StructField(NotificationEmails,StructType(List(StructField(GettingStartedEmails,ArrayType(StringType,true),true),StructField(RenewalAndTrialEmails,ArrayType(StringType,true),true),StructField(ChannelsAndAuthorizationEmails,ArrayType(StringType,true),true),StructField(SupportEmails,ArrayType(StringType,true),true),StructField(BillingEmails,ArrayType(StringType,true),true),StructField(SignatureEmail,ArrayType(StringType,true),true))),true),StructField(BrandCustomization,StructType(List(StructField(ProfileLogoExternalFileName,StringType,true),StructField(HeaderBackgroundColor,StringType,true),StructField(HeaderTextColor,StringType,true),StructField(SeparatorLineColor,StringType,true))),true),StructField(StatementSettings,StructType(List(StructField(SharedStatementType,IntegerType,true),StructField(DefaultRemark,StringType,true))),true)))

#### BUILD & VERIFY Non-Parsed DF w JSON/Array of JSONs as string

In [11]:
# BUILD & VERIFY Non-Parsed DF
df = spark.createDataFrame(data, ("key", "value"))
display(df)

key,value
1,"{""CompanyProfile"":{""PrimaryAddress"":{""AddressLine1"":""5001 Statesman Dr"",""AddressLine2"":"""",""City"":""Irving"",""State"":"""",""Country"":""US"",""ZipCode"":""75063"",""County"":""county save""},""ProfileBillingAddress"":{""AddressLine1"":""5001 Statesman Dr"",""AddressLine2"":"""",""City"":""Irving"",""State"":"""",""Country"":""US"",""ZipCode"":""75063"",""County"":""county save""},""BillingPhone"":""7222567865"",""BillingEmail"":""sowmya.simhadri@techdata.com"",""ProfilePhone"":""7222567864"",""ProfileFaxNumber"":""7222563375"",""ProfileEmail"":""sowmya.simhadri@techdata.com"",""ConsolidationType"":1,""BillingAddressType"":1},""NotificationEmails"":{""GettingStartedEmails"":[""sowmya.simhadri@techdata.com""],""RenewalAndTrialEmails"":[""sowmya.simhadri@techdata.com""],""ChannelsAndAuthorizationEmails"":[""sowmya.simhadri@techdata.com""],""SupportEmails"":[""sowmya.simhadri@techdata.com""],""BillingEmails"":[""sowmya.simhadri@techdata.com""],""SignatureEmail"":[""sowmya.simhadri@techdata.com""]},""BrandCustomization"":{""ProfileLogoExternalFileName"":"""",""HeaderBackgroundColor"":""#aaebff"",""HeaderTextColor"":""#aaebff"",""SeparatorLineColor"":""#aaebff""},""StatementSettings"":{""SharedStatementType"":1,""DefaultRemark"":""""}}"


In [12]:
# BUILD & VERIFY Parsed DF
# VERIFY SCHEMA enforcement works or not
df1 = df.withColumn("value_w_schema", psf.from_json(df.value, schema2["profilesetting"]))
df1.select("value_w_schema").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
value_w_schema |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[[[5001 Statesman Dr, , Irving, , US, 75063, county save], [5001 Statesman Dr, , Irving, , US, 75063, county save], 7222567865, sowmya.simhadri@techdata.com, 7222567864, 7222563375, sowmya.simhadri@techdata.com, 1, 1], [[sowmya.simhadri@techdata.com], [sowmya.simhadri@techdata.com], [sowmya.simhadri@techdata.com], [sowmya.simhadri@techdata.com], [sowmya.simhadri@techdata.com], [sowmya.simhadri@techdata.com]], [, #aaebff, #aaebff, #aaebff], [1, ]]|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [13]:
# VERIFY Before FLATTEN
df1.printSchema()
df1.dtypes

root
-- key: long (nullable = true)
-- value: string (nullable = true)
-- value_w_schema: struct (nullable = true)
 |-- CompanyProfile: struct (nullable = true)
 | |-- PrimaryAddress: struct (nullable = true)
 | | |-- AddressLine1: string (nullable = true)
 | | |-- AddressLine2: string (nullable = true)
 | | |-- City: string (nullable = true)
 | | |-- State: string (nullable = true)
 | | |-- Country: string (nullable = true)
 | | |-- ZipCode: string (nullable = true)
 | | |-- County: string (nullable = true)
 | |-- ProfileBillingAddress: struct (nullable = true)
 | | |-- AddressLine1: string (nullable = true)
 | | |-- AddressLine2: string (nullable = true)
 | | |-- City: string (nullable = true)
 | | |-- State: string (nullable = true)
 | | |-- Country: string (nullable = true)
 | | |-- ZipCode: string (nullable = true)
 | | |-- County: string (nullable = true)
 | |-- BillingPhone: string (nullable = true)
 | |-- BillingEmail: string (nullable = true)
 | |-- ProfilePhone: string (nullable = true)
 | |-- ProfileFaxNumber: string (nullable = true)
 | |-- ProfileEmail: string (nullable = true)
 | |-- ConsolidationType: integer (nullable = true)
 | |-- BillingAddressType: integer (nullable = true)
 |-- NotificationEmails: struct (nullable = true)
 | |-- GettingStartedEmails: array (nullable = true)
 | | |-- element: string (containsNull = true)
 | |-- RenewalAndTrialEmails: array (nullable = true)
 | | |-- element: string (containsNull = true)
 | |-- ChannelsAndAuthorizationEmails: array (nullable = true)
 | | |-- element: string (containsNull = true)
 | |-- SupportEmails: array (nullable = true)
 | | |-- element: string (containsNull = true)
 | |-- BillingEmails: array (nullable = true)
 | | |-- element: string (containsNull = true)
 | |-- SignatureEmail: array (nullable = true)
 | | |-- element: string (containsNull = true)
 |-- BrandCustomization: struct (nullable = true)
 | |-- ProfileLogoExternalFileName: string (nullable = true)
 | |-- HeaderBackgroundColor: string (nullable = true)
 | |-- HeaderTextColor: string (nullable = true)
 | |-- SeparatorLineColor: string (nullable = true)
 |-- StatementSettings: struct (nullable = true)
 | |-- SharedStatementType: integer (nullable = true)
 | |-- DefaultRemark: string (nullable = true)

Out[13]: [('key', 'bigint'),
 ('value', 'string'),
 ('value_w_schema',
 'struct<CompanyProfile:struct<PrimaryAddress:struct<AddressLine1:string,AddressLine2:string,City:string,State:string,Country:string,ZipCode:string,County:string>,ProfileBillingAddress:struct<AddressLine1:string,AddressLine2:string,City:string,State:string,Country:string,ZipCode:string,County:string>,BillingPhone:string,BillingEmail:string,ProfilePhone:string,ProfileFaxNumber:string,ProfileEmail:string,ConsolidationType:int,BillingAddressType:int>,NotificationEmails:struct<GettingStartedEmails:array<string>,RenewalAndTrialEmails:array<string>,ChannelsAndAuthorizationEmails:array<string>,SupportEmails:array<string>,BillingEmails:array<string>,SignatureEmail:array<string>>,BrandCustomization:struct<ProfileLogoExternalFileName:string,HeaderBackgroundColor:string,HeaderTextColor:string,SeparatorLineColor:string>,StatementSettings:struct<SharedStatementType:int,DefaultRemark:string>>')]

### 3rd Set of Test Data - Order . Subs

In [15]:
# Raw data from Landing
# ---------------------
csvFilePath = "/FileStore/tables/order_subs_5_row-d6249.csv"

order_subs_df = (spark.read
  .format("csv")
  .option("header", True)
  .option("delimiter", '\t')
  .option("inferSchema", True)
  .load(csvFilePath)
)
display(order_subs_df)

SubscriptionId,OrderId,OrderKey,SubscriptionKey,ResellerKey,ParentSubscriptionKey,VendorSubscriptionKey,ParentVendorSubscriptionKey,CustomerKey,ProductKey,ProductName,ServiceName,VendorKey,PlanKey,PlanName,ServiceKey,VendorServiceKey,VendorSubscriptionDisplayName,VendorProvisionResponse,SubscriptionStatus,ProvisioningStatus,Quantity,Price,VendorErrorMessage,IsNewCustomer,UseDefaultSubscriptionName,LastProvisionRequestType,CreatedOnUtc,UpdatedOnUtc,UnitType,CreatedBy,UpdatedBy,BillingFrequency,BillingType,PaymentMethodType,CardId,IsSandBox,OrderPONumber,OrderDate,VendorAccount,BillingEndDate,AccessInformation,Trial,IsTrialSelected,OldQuantity,RemainingVendorSettingsName,ResellerSettings,CustomerSettings,ProductSettings,Services,Domain,ParentSolutionSubscriptionKey,SolutionKey,TenantId,VertexTransactionId,ProfileId,MarketplaceKey,RateCardId,CustomerUserId,AzureUsers,IsBeingProcessed,CustomerCredentials,FloorPlanId,ActivationDate,ProvisionDate,CanResellerReceiveCredentials,CanVendorReceiveInformation,RegionCode,Currency,PartnerId,OldSubscriptionStatus,DelegateJson,ProvisionedBy,Account,AddOnServices,IsMigrationOrder,RateCardVersion,PendingDeploymentNotificationFlag,PlanUpgradeKey
0001228a-9ca2-4402-8725-b71f52438723,71fa9400-66d3-435d-bcc5-907c77401682,S125051827,c84b9bfa-c0f7-4763-82bf-ecd5b1059dac,0000470122,NULL,NULL,NULL,ef40d8e7-7f47-455a-94a3-21e5ab9d8fde,PDCGKXQNTK98TE,SS FEB 3 MS (N) BE Reseller,NULL,microsoft,PLP43GJQHV3RH3,Staff - E1 Annual - N,NULL,NULL,NULL,NULL,0,1,1.0,40.0,NULL,NULL,NULL,0,2020-03-15T03:05:06.362+0000,2020-03-15T03:05:06.362+0000,NULL,50D88FC3-ABB6-4EAE-92B2-3D1027E84D81,NULL,1,1,1,NULL,NULL,S125051827,2020-03-15T03:05:06.339+0000,NULL,NULL,NULL,NULL,NULL,1.0,NULL,NULL,NULL,NULL,"[{""ServiceKey"":""SVEFOTLGK5DFAH"",""VendorServiceKey"":""91fd106f-4b2c-4938-95ac-f54f74e9a239"",""VendorSubscriptionKey"":null,""Name"":""Office 365 Enterprise E1 - Annual BE- Nested"",""VendorProvisionResponse"":null,""ProvisionStatus"":1,""SubscriptionStatus"":0,""BillingFrequency"":1,""CreatedOnUtc"":null,""UpdatedOnUtc"":null,""Quantity"":1.0,""Price"":20.0,""RateCardId"":1012037,""RateCardVersion"":""2020.02.001"",""Margin"":0.0,""DefaultQuantity"":null,""Cost"":20.0,""ProvisionDate"":null,""ParentServiceKey"":null,""ServiceConfiguration"":null,""IsDummy"":null,""VendorKey"":null,""EULADateTime"":null}]",NULL,NULL,NULL,NULL,NULL,1204,BE,NULL,0730a94b-5049-4db6-88a8-e8807774a3b5,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,EUR,2316707,0,NULL,NULL,NULL,NULL,2,NULL,NULL,NULL
00012ec4-0de8-41d9-85e3-85e8bf46c8a2,de9fe059-4e8c-4606-b613-7c7e6f5ed827,S125032949,40808027-2d73-41d3-a96a-edbe5f02b578,techdata,NULL,NULL,NULL,6c6d1ae2-2509-4b6b-becc-702a6fb8cc7e,PDDADPYMDNJO8J,QATeam MS Reseller,NULL,microsoft,PLTPTZUMTYT6NX,Staff - Microsoft OneTime,NULL,NULL,NULL,NULL,0,1,1.0,10.006,NULL,NULL,NULL,0,2020-03-11T12:44:46.522+0000,2020-03-11T12:44:46.522+0000,NULL,1BEF951C-11FF-44EF-A6C7-D70EA43F972F,NULL,2,1,1,NULL,NULL,567567567,2020-03-11T12:44:46.347+0000,NULL,NULL,NULL,NULL,NULL,1.0,NULL,NULL,NULL,NULL,"[{""ServiceKey"":""SVNLXSFEQFXUHM"",""VendorServiceKey"":""ms-quattro-one-time-abc"",""VendorSubscriptionKey"":null,""Name"":""Quatro Service - One Time"",""VendorProvisionResponse"":null,""ProvisionStatus"":1,""SubscriptionStatus"":0,""BillingFrequency"":2,""CreatedOnUtc"":null,""UpdatedOnUtc"":null,""Quantity"":1.0,""Price"":10.006,""RateCardId"":1011923,""RateCardVersion"":null,""Margin"":0.0,""DefaultQuantity"":null,""Cost"":10.006,""ProvisionDate"":null,""ParentServiceKey"":null,""ServiceConfiguration"":null,""IsDummy"":null,""VendorKey"":null,""EULADateTime"":null}]",NULL,NULL,NULL,NULL,NULL,201,US,NULL,85f96a10-1c11-48ce-af8c-5f31a93ac435,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,USD,null,0,NULL,NULL,NULL,NULL,2,NULL,NULL,NULL
00014ae0-3e5f-45f7-a7b7-105130794fe0,a00e4a72-a164-467d-86f1-4c1d10c96aca,S125057544,77f5bc91-b686-42a5-8aae-1777fc356453,0000685851,NULL,NULL,NULL,f77793ae-2747-436f-8a5f-73bab125c44b,PDW0BS

In [16]:
subscription_services_schema = ArrayType(StructType([
  StructField("ServiceKey", StringType()),
  StructField("VendorServiceKey", StringType()),
  StructField("VendorSubscriptionKey", StringType()),
  StructField("Name", StringType()),
  StructField("VendorProvisionResponse", StringType()),
  StructField("ProvisionStatus", IntegerType()),
  StructField("SubscriptionStatus", IntegerType()),
  StructField("BillingFrequency", IntegerType()),
  StructField("CreatedOnUtc", StringType()),
  StructField("UpdatedOnUtc", StringType()),
  StructField("Quantity", FloatType()),
  StructField("Price", FloatType()),
  StructField("RateCardId", LongType()),
  StructField("RateCardVersion", StringType()),
  StructField("Margin", FloatType()),
  StructField("DefaultQuantity", StringType()),
  StructField("Cost", FloatType()),
  StructField("ProvisionDate", StringType()),
  StructField("ParentServiceKey", StringType()),
  StructField("ServiceConfiguration", StringType()),
  StructField("IsDummy", StringType()),
  StructField("VendorKey", StringType()),
  StructField("EULADateTime", StringType())
]))

In [17]:
# Enforce Schema on String JSON
# ------------------------------
df = order_subs_df.withColumn("Services_parsed", psf.from_json(psf.col("Services"), subscription_services_schema))
df.dtypes
df.select("Services_parsed").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
Services_parsed |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[[SVEFOTLGK5DFAH, 91fd106f-4b2c-4938-95ac-f54f74e9a239,, Office 365 Enterprise E1 - Annual BE- Nested,, 1, 0, 1,,, 1.0, 20.0, 1012037, 2020.02.001, 0.0,, 20.0,,,,,,]] |
[[SVNLXSFEQFXUHM, ms-quattro-one-time-abc,, Quatro Service - One Time,, 1, 0, 2,,, 1.0, 10.006, 1011923,, 0.0,, 10.006,,,,,,]] |
[[SVPU2O81V1TTJE, 91fd106f-4b2c-4938-95ac-f54f74e9a239, 7918AACA-D4DC-4DF5-A5EB-E62930731F9E, Office 365 Enterprise E1 - BE,, 4, 1, 1, 2020-03-17T05:31:44.380385Z, 2020-03-17T05:31:44.3803872Z, 2.0, 16.5, 1012056,, 50.0, 1.0, 11.0,,,,,,]] |
[[SVOBRWEAJI5UT8, 91fd106f-4b2c-4938-95ac-f54f74e9a239,, Office 365 Enterprise E1 - Annual BE,, 1, 0, 2,,, 1.0, 48.0, 1012004,, 100.0,, 24.0,,,,,,]] |
[[SVARRAEGMIHYSD, 91fd106f-4b2c-4938-95ac-f54f74e9a239, 756de476-ead5-4923-ac56-af1ef83d337a, Office 365 Enterprise E1 - BE- Nested,, 4, 1, 1, 2020-03-22T18:49:14.3299019Z, 2020-03-22T18:49:14.3299041Z, 1.0, 5.0, 1012494, 2020.02.001, 0.0,, 5.0,,,,,,]]|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

### Call Struct Column flattener - Just Pass a DF, the recursive function will flatten the whole DF

In [19]:
# CALL Struct Flattener Fn:
# User inputs to Fn
# schema enforced df name = df1
#pk = ["key"] # Not needed
#json_col_name = "value_w_schema" #Not needed
#-----------------------------------------------------------------
fully_flat_df = recursive_struct_flattener(df)

--------- START recursive_struct_flattener 1 ----------
--------- START recursive_struct_flattener 1 ----------
flat fields: 
['SubscriptionId', 'OrderId', 'OrderKey', 'SubscriptionKey', 'ResellerKey', 'ParentSubscriptionKey', 'VendorSubscriptionKey', 'ParentVendorSubscriptionKey', 'CustomerKey', 'ProductKey', 'ProductName', 'ServiceName', 'VendorKey', 'PlanKey', 'PlanName', 'ServiceKey', 'VendorServiceKey', 'VendorSubscriptionDisplayName', 'VendorProvisionResponse', 'SubscriptionStatus', 'ProvisioningStatus', 'Quantity', 'Price', 'VendorErrorMessage', 'IsNewCustomer', 'UseDefaultSubscriptionName', 'LastProvisionRequestType', 'CreatedOnUtc', 'UpdatedOnUtc', 'UnitType', 'CreatedBy', 'UpdatedBy', 'BillingFrequency', 'BillingType', 'PaymentMethodType', 'CardId', 'IsSandBox', 'OrderPONumber', 'OrderDate', 'VendorAccount', 'BillingEndDate', 'AccessInformation', 'Trial', 'IsTrialSelected', 'OldQuantity', 'RemainingVendorSettingsName', 'ResellerSettings', 'CustomerSettings', 'ProductSettings', 'Services', 'Domain', 'ParentSolutionSubscriptionKey', 'SolutionKey', 'TenantId', 'VertexTransactionId', 'ProfileId', 'MarketplaceKey', 'RateCardId', 'CustomerUserId', 'AzureUsers', 'IsBeingProcessed', 'CustomerCredentials', 'FloorPlanId', 'ActivationDate', 'ProvisionDate', 'CanResellerReceiveCredentials', 'CanVendorReceiveInformation', 'RegionCode', 'Currency', 'PartnerId', 'OldSubscriptionStatus', 'DelegateJson', 'ProvisionedBy', 'Account', 'AddOnServices', 'IsMigrationOrder', 'RateCardVersion', 'PendingDeploymentNotificationFlag', 'PlanUpgradeKey']
Struct fields: 
['Services_parsed_exploded']
Array or Map fields: 
[]
completed flattening of Struct in DF column 
Calling self recursively

#### Verify DF flattening Section

In [21]:
#Verify Types AFTER Flatten
fully_flat_df.dtypes

Out[16]: [('SubscriptionId', 'string'),
 ('OrderId', 'string'),
 ('OrderKey', 'string'),
 ('SubscriptionKey', 'string'),
 ('ResellerKey', 'string'),
 ('ParentSubscriptionKey', 'string'),
 ('VendorSubscriptionKey', 'string'),
 ('ParentVendorSubscriptionKey', 'string'),
 ('CustomerKey', 'string'),
 ('ProductKey', 'string'),
 ('ProductName', 'string'),
 ('ServiceName', 'string'),
 ('VendorKey', 'string'),
 ('PlanKey', 'string'),
 ('PlanName', 'string'),
 ('ServiceKey', 'string'),
 ('VendorServiceKey', 'string'),
 ('VendorSubscriptionDisplayName', 'string'),
 ('VendorProvisionResponse', 'string'),
 ('SubscriptionStatus', 'int'),
 ('ProvisioningStatus', 'int'),
 ('Quantity', 'double'),
 ('Price', 'double'),
 ('VendorErrorMessage', 'string'),
 ('IsNewCustomer', 'string'),
 ('UseDefaultSubscriptionName', 'string'),
 ('LastProvisionRequestType', 'int'),
 ('CreatedOnUtc', 'timestamp'),
 ('UpdatedOnUtc', 'timestamp'),
 ('UnitType', 'string'),
 ('CreatedBy', 'string'),
 ('UpdatedBy', 'string'),
 ('BillingFrequency', 'int'),
 ('BillingType', 'int'),
 ('PaymentMethodType', 'int'),
 ('CardId', 'string'),
 ('IsSandBox', 'string'),
 ('OrderPONumber', 'string'),
 ('OrderDate', 'timestamp'),
 ('VendorAccount', 'string'),
 ('BillingEndDate', 'string'),
 ('AccessInformation', 'string'),
 ('Trial', 'string'),
 ('IsTrialSelected', 'string'),
 ('OldQuantity', 'double'),
 ('RemainingVendorSettingsName', 'string'),
 ('ResellerSettings', 'string'),
 ('CustomerSettings', 'string'),
 ('ProductSettings', 'string'),
 ('Services', 'string'),
 ('Domain', 'string'),
 ('ParentSolutionSubscriptionKey', 'string'),
 ('SolutionKey', 'string'),
 ('TenantId', 'string'),
 ('VertexTransactionId', 'string'),
 ('ProfileId', 'int'),
 ('MarketplaceKey', 'string'),
 ('RateCardId', 'string'),
 ('CustomerUserId', 'string'),
 ('AzureUsers', 'string'),
 ('IsBeingProcessed', 'string'),
 ('CustomerCredentials', 'string'),
 ('FloorPlanId', 'string'),
 ('ActivationDate', 'string'),
 ('ProvisionDate', 'string'),
 ('CanResellerReceiveCredentials', 'string'),
 ('CanVendorReceiveInformation', 'string'),
 ('RegionCode', 'string'),
 ('Currency', 'string'),
 ('PartnerId', 'int'),
 ('OldSubscriptionStatus', 'string'),
 ('DelegateJson', 'string'),
 ('ProvisionedBy', 'string'),
 ('Account', 'string'),
 ('AddOnServices', 'string'),
 ('IsMigrationOrder', 'int'),
 ('RateCardVersion', 'string'),
 ('PendingDeploymentNotificationFlag', 'string'),
 ('PlanUpgradeKey', 'string'),
 ('Services_parsed_exploded_ServiceKey', 'string'),
 ('Services_parsed_exploded_VendorServiceKey', 'string'),
 ('Services_parsed_exploded_VendorSubscriptionKey', 'string'),
 ('Services_parsed_exploded_Name', 'string'),
 ('Services_parsed_exploded_VendorProvisionResponse', 'string'),
 ('Services_parsed_exploded_ProvisionStatus', 'int'),
 ('Services_parsed_exploded_SubscriptionStatus', 'int'),
 ('Services_parsed_exploded_BillingFrequency', 'int'),
 ('Services_parsed_exploded_CreatedOnUtc', 'string'),
 ('Services_parsed_exploded_UpdatedOnUtc', 'string'),
 ('Services_parsed_exploded_Quantity', 'float'),
 ('Services_parsed_exploded_Price', 'float'),
 ('Services_parsed_exploded_RateCardId', 'bigint'),
 ('Services_parsed_exploded_RateCardVersion', 'string'),
 ('Services_parsed_exploded_Margin', 'float'),
 ('Services_parsed_exploded_DefaultQuantity', 'string'),
 ('Services_parsed_exploded_Cost', 'float'),
 ('Services_parsed_exploded_ProvisionDate', 'string'),
 ('Services_parsed_exploded_ParentServiceKey', 'string'),
 ('Services_parsed_exploded_ServiceConfiguration', 'string'),
 ('Services_parsed_exploded_IsDummy', 'string'),
 ('Services_parsed_exploded_VendorKey', 'string'),
 ('Services_parsed_exploded_EULADateTime', 'string')]

In [22]:
#Verify Values AFTER Flatten
display(fully_flat_df)

SubscriptionId,OrderId,OrderKey,SubscriptionKey,ResellerKey,ParentSubscriptionKey,VendorSubscriptionKey,ParentVendorSubscriptionKey,CustomerKey,ProductKey,ProductName,ServiceName,VendorKey,PlanKey,PlanName,ServiceKey,VendorServiceKey,VendorSubscriptionDisplayName,VendorProvisionResponse,SubscriptionStatus,ProvisioningStatus,Quantity,Price,VendorErrorMessage,IsNewCustomer,UseDefaultSubscriptionName,LastProvisionRequestType,CreatedOnUtc,UpdatedOnUtc,UnitType,CreatedBy,UpdatedBy,BillingFrequency,BillingType,PaymentMethodType,CardId,IsSandBox,OrderPONumber,OrderDate,VendorAccount,BillingEndDate,AccessInformation,Trial,IsTrialSelected,OldQuantity,RemainingVendorSettingsName,ResellerSettings,CustomerSettings,ProductSettings,Services,Domain,ParentSolutionSubscriptionKey,SolutionKey,TenantId,VertexTransactionId,ProfileId,MarketplaceKey,RateCardId,CustomerUserId,AzureUsers,IsBeingProcessed,CustomerCredentials,FloorPlanId,ActivationDate,ProvisionDate,CanResellerReceiveCredentials,CanVendorReceiveInformation,RegionCode,Currency,PartnerId,OldSubscriptionStatus,DelegateJson,ProvisionedBy,Account,AddOnServices,IsMigrationOrder,RateCardVersion,PendingDeploymentNotificationFlag,PlanUpgradeKey,Services_parsed_exploded_ServiceKey,Services_parsed_exploded_VendorServiceKey,Services_parsed_exploded_VendorSubscriptionKey,Services_parsed_exploded_Name,Services_parsed_exploded_VendorProvisionResponse,Services_parsed_exploded_ProvisionStatus,Services_parsed_exploded_SubscriptionStatus,Services_parsed_exploded_BillingFrequency,Services_parsed_exploded_CreatedOnUtc,Services_parsed_exploded_UpdatedOnUtc,Services_parsed_exploded_Quantity,Services_parsed_exploded_Price,Services_parsed_exploded_RateCardId,Services_parsed_exploded_RateCardVersion,Services_parsed_exploded_Margin,Services_parsed_exploded_DefaultQuantity,Services_parsed_exploded_Cost,Services_parsed_exploded_ProvisionDate,Services_parsed_exploded_ParentServiceKey,Services_parsed_exploded_ServiceConfiguration,Services_parsed_exploded_IsDummy,Services_parsed_exploded_VendorKey,Services_parsed_exploded_EULADateTime
0001228a-9ca2-4402-8725-b71f52438723,71fa9400-66d3-435d-bcc5-907c77401682,S125051827,c84b9bfa-c0f7-4763-82bf-ecd5b1059dac,0000470122,NULL,NULL,NULL,ef40d8e7-7f47-455a-94a3-21e5ab9d8fde,PDCGKXQNTK98TE,SS FEB 3 MS (N) BE Reseller,NULL,microsoft,PLP43GJQHV3RH3,Staff - E1 Annual - N,NULL,NULL,NULL,NULL,0,1,1.0,40.0,NULL,NULL,NULL,0,2020-03-15T03:05:06.362+0000,2020-03-15T03:05:06.362+0000,NULL,50D88FC3-ABB6-4EAE-92B2-3D1027E84D81,NULL,1,1,1,NULL,NULL,S125051827,2020-03-15T03:05:06.339+0000,NULL,NULL,NULL,NULL,NULL,1.0,NULL,NULL,NULL,NULL,"[{""ServiceKey"":""SVEFOTLGK5DFAH"",""VendorServiceKey"":""91fd106f-4b2c-4938-95ac-f54f74e9a239"",""VendorSubscriptionKey"":null,""Name"":""Office 365 Enterprise E1 - Annual BE- Nested"",""VendorProvisionResponse"":null,""ProvisionStatus"":1,""SubscriptionStatus"":0,""BillingFrequency"":1,""CreatedOnUtc"":null,""UpdatedOnUtc"":null,""Quantity"":1.0,""Price"":20.0,""RateCardId"":1012037,""RateCardVersion"":""2020.02.001"",""Margin"":0.0,""DefaultQuantity"":null,""Cost"":20.0,""ProvisionDate"":null,""ParentServiceKey"":null,""ServiceConfiguration"":null,""IsDummy"":null,""VendorKey"":null,""EULADateTime"":null}]",NULL,NULL,NULL,NULL,NULL,1204,BE,NULL,0730a94b-5049-4db6-88a8-e8807774a3b5,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,EUR,2316707,0,NULL,NULL,NULL,NULL,2,NULL,NULL,NULL,SVEFOTLGK5DFAH,91fd106f-4b2c-4938-95ac-f54f74e9a239,null,Office 365 Enterprise E1 - Annual BE- Nested,null,1,0,1,null,null,1.0,20.0,1012037,2020.02.001,0.0,null,20.0,null,null,null,null,null,null
00012ec4-0de8-41d9-85e3-85e8bf46c8a2,de9fe059-4e8c-4606-b613-7c7e6f5ed827,S125032949,40808027-2d73-41d3-a96a-edbe5f02b578,techdata,NULL,NULL,NULL,6c6d1ae2-2509-4b6b-becc-702a6fb8cc7e,PDDADPYMDNJO8J,QATeam MS Reseller,NULL,microsoft,PLTPTZUMTYT6NX,Staff - Microsoft OneTime,NULL,NULL,NULL,NULL,0,1,1.0,10.006,NULL,NULL,NULL,0,2020-03-11T12:44:46.522+0000,2020-03-11T12:44:46.522+0000,NULL,1BEF

###### End